In [ ]:
# скачиваем файл
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VPkoWfiIvZl4HGp49BUaVEEblVIGYh91' -O heart.csv

--2025-02-08 13:51:57--  https://docs.google.com/uc?export=download&id=1VPkoWfiIvZl4HGp49BUaVEEblVIGYh91
Resolving docs.google.com (docs.google.com)... 173.194.206.113, 173.194.206.139, 173.194.206.102, ...
Connecting to docs.google.com (docs.google.com)|173.194.206.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1VPkoWfiIvZl4HGp49BUaVEEblVIGYh91&export=download [following]
--2025-02-08 13:51:57--  https://drive.usercontent.google.com/download?id=1VPkoWfiIvZl4HGp49BUaVEEblVIGYh91&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.126.132, 2607:f8b0:4001:c1d::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.126.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11328 (11K) [application/octet-stream]
Saving to: ‘heart.csv’

heart.csv           100%[===================>]  11.06K  --.-KB/s    in 0s

In [ ]:
# устанавливаем нужную версию библиотеки sklearn, чтобы ответы сошлись
!pip install scikit-learn==0.22.1

  Using cached scikit-learn-0.22.1.tar.gz (6.9 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for scikit-learn
  Running setup.py clean for scikit-learn
Failed to build scikit-learn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (scikit-learn)


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
data.groupby('target').size() / len(data)

,0
target,
0,0.455446
1,0.544554


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.25, random_state=13)
X_train.shape, X_test.shape


((227, 13), (76, 13))

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(max_iter=1000, learning_rate='constant', eta0=0.1, random_state=13, alpha  = 0, loss='log_loss')
# заметьте - в данном случае добавлять единичный признак в датасет не нужно, потому что метод SGDClassifier создаст его сам
clf.fit(X_train, y_train)
print(clf.intercept_)



[-1.15927929]


In [ ]:
arr = clf.coef_[0]
print(arr)
print(round(np.linalg.norm(arr), 2))

sum1 = (arr[:]**2).sum()
print((sum1)**(1/2))

summ1 = 0
for i in arr:
  #print(i)
  summ1 += i**2
print(summ1**(1/2))

[-93.18252315 -18.25927929  33.13144142 -65.99603819   1.75342142
   2.24072071   6.44072071 112.79675487 -16.1        -33.35463964
  11.73144141 -38.3        -25.37783787]
175.68
175.67735435821564
175.67735435821564


In [ ]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.618421052631579

In [ ]:
clf_2 = SGDClassifier(max_iter=1000, random_state=13, alpha  = 0.1, penalty='l1',learning_rate='optimal', loss = 'log_loss')
# заметьте - в данном случае добавлять единичный признак в датасет не нужно, потому что метод SGDClassifier создаст его сам
clf_2.fit(X_train, y_train)



SGDClassifier(alpha=0.1, loss='log_loss', penalty='l1', random_state=13)

In [ ]:
y_pred = clf_2.predict(X_test)
accuracy_score(y_test, y_pred)

0.631578947368421

In [ ]:
arr = clf_2.coef_[0]
print(arr)
print(max(arr))
print(round(np.linalg.norm(arr), 2))
arr = clf_2.coef_[0]
sum1 = (arr[:]**2).sum()
print((sum1)**(1/2))

summ1 = 0
for i in arr:
  #print(i)
  summ1 += i**2
print(summ1**(1/2))

[-8.21921343e-01  0.00000000e+00  9.22250690e+00 -3.99493936e-01
 -8.00556930e-03  0.00000000e+00  0.00000000e+00  5.02927853e-01
  0.00000000e+00 -1.80963290e-01  0.00000000e+00 -4.07015387e+00
 -2.07554492e+00]
9.222506904181394
10.35
10.34647626637087
10.34647626637087


In [ ]:
y_pred_prob = clf.predict_proba(X_test)[:, 1]


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred_prob)

0.5697674418604651

In [ ]:
y_pred_prob = clf_2.predict_proba(X_test)[:, 1]
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred_prob)

0.8477801268498942